## Classic Machine Learning Algorithms from Scratch

This notebook implements several foundational machine learning algorithms from scratch using NumPy and matplotlib — without relying on external machine learning libraries like scikit-learn.  
It is designed as an educational exercise to understand how these algorithms work under the hood.


# **Part A - SMO Algorithm for SVM**

In this section, we implement the SMO (Sequential Minimal Optimization) algorithm used to train a Support Vector Machine (SVM).  
The SMO algorithm optimizes the dual form of the SVM problem by updating pairs of Lagrange multipliers (`alpha`) iteratively.

We support both a linear kernel and an RBF (Radial Basis Function) kernel.  
The algorithm includes kernel matrix precomputation, KKT condition checks, alpha updates, clipping, and bias calculation.

In [1]:
import numpy as np

In [2]:
# Linear kernel function
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

# RBF kernel functions
def rbf_kernel(x1, x2, gamma):
    return np.exp(-gamma * np.linalg.norm(x1 - x2)**2)

# Compute the error for a given sample i, can use linear and RBF kernel
def compute_error(i, X, y, a, b, kernel_matrix):
    return np.dot(a * y, kernel_matrix[:, i]) + b - y[i]

# Clips the value of alpha to stay within the range [L, H]
def clip_alpha(a, L, H):
    if a <= L:
        return L
    elif a >= H:
        return H
    else:
        return a

In [3]:
# SMO algorithm
def smo_algorithm(X, y, C, tol=1e-3, max_iter=1000, kernel=linear_kernel):
    """
    Trains a Support Vector Machine (SVM) using the Sequential Minimal Optimization (SMO) algorithm.

    Parameters:
        X (ndarray): Training features of shape (n_samples, n_features)
        y (ndarray): Training labels of shape (n_samples,), with values in {-1, 1}
        C (float): Regularization parameter
        tol (float): Tolerance for stopping criterion
        max_iter (int): Maximum number of iterations
        kernel (function): Kernel function to compute similarity between samples

    Returns:
        a (ndarray): Optimized Lagrange multipliers (alphas)
        b (float): Bias term of the decision boundary
    """
    n_samples, n_features = X.shape
    a = np.zeros(n_samples)  # Initialize Lagrange multipliers
    b = 0  # Bias term
    iter_count = 0  # Count total iterations
    no_change_iter_count = 0  # Iterations without alpha changes

    # Precompute the kernel matrix
    kernel_matrix = np.array([[kernel(X[i], X[j]) for j in range(n_samples)] for i in range(n_samples)])

    while iter_count < max_iter:
        a_changed = 0  # Track number of alpha updates in this iteration

        for i in range(n_samples):
            E1 = compute_error(i, X, y, a, b, kernel_matrix)

            # Check if the sample violates KKT conditions
            if (y[i] * E1 < -tol and a[i] < C) or (y[i] * E1 > tol and a[i] > 0):

                # Select j by maximize |E1 - E2|
                E = np.array([compute_error(k, X, y, a, b, kernel_matrix) for k in range(n_samples)])
                j_c = np.argsort(-np.abs(E - E1))  # Descending order of errors
                j = j_c[1] if j_c[0] == i else j_c[0] # Choose the second largest if the first is i

                E2 = compute_error(j, X, y, a, b, kernel_matrix)
                a1_old, a2_old = a[i], a[j] # keep the old Lagrange multipliers

                # Compute L and H for clipping
                if y[i] != y[j]: # use for diffrent labels (s!=1)
                    L = max(0, a[j] - a[i])
                    H = min(C, C + a[j] - a[i])
                else: # for same labels (s=1)
                    L = max(0, a[i] + a[j] - C)
                    H = min(C, a[i] + a[j])

                if L == H:
                    continue

                # Compute eta
                eta = kernel_matrix[i, i] + kernel_matrix[j, j] - 2 * kernel_matrix[i, j]
                if eta == 0:
                    continue

                # Update alpha[j]
                a[j] += y[j] * (E1 - E2) / eta
                a[j] = clip_alpha(a[j], L, H) # apdate a2 by clipping

                # Check for significant change in alpha[j]
                if abs(a[j] - a2_old) < tol:
                    continue

                # Update alpha[i] based on alpha[j] change
                a[i] += y[i] * y[j] * (a2_old - a[j])

                # Compute new bias terms
                b1 = b - E1 - y[i] * (a[i] - a1_old) * kernel_matrix[i, i] - y[j] * (a[j] - a2_old) * kernel_matrix[i, j]
                b2 = b - E2 - y[i] * (a[i] - a1_old) * kernel_matrix[i, j] - y[j] * (a[j] - a2_old) * kernel_matrix[j, j]

                # Update the bias term based on support vectors
                if 0 < a[i] < C:
                    b = b1
                elif 0 < a[j] < C:
                    b = b2
                else:
                    b = (b1 + b2) / 2  # Select the average of b1 and b2

                a_changed += 1  # Count changes

        # Update sum of iteration
        iter_count += 1

        # Update iteration count if no changes
        no_change_iter_count = no_change_iter_count + 1 if a_changed == 0 else 0

        # Stop after 10 consecutive iterations without change
        if no_change_iter_count >= 10:
            break

    return a, b

# **Part B - SVM Training & Evaluation**

In this section, we apply the SMO algorithm to train binary SVM classifiers using both linear and RBF kernels on the Iris dataset.

For each class in the dataset, we:
- Relabel the data for one-vs-rest classification
- Split the data into training, validation, and test sets
- Tune hyperparameters (C and gamma) using validation accuracy
- Train final models with best-found parameters
- Evaluate each model on the test set using accuracy, confusion matrix, and sensitivity

The results are printed per class and per kernel.

In [4]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from tqdm import tqdm

In [5]:
# Predict function
def predict(X_train, y_train, X_test, a, b, kernel):
    # Compute kernel matrix
    kernel_matrix = np.array([[kernel(x_train, x_test) for x_test in X_test] for x_train in X_train])
    # Compute decisions
    decision = np.dot(kernel_matrix.T, (a * y_train)) + b

    return np.sign(decision)


# Hyperparameter tuning function for C and gamma
def tune_parameters(X_train, y_train, X_val, y_val, C_values, gamma_values=None, kernel_type=None):
    """
    Performs hyperparameter tuning for an SVM using validation accuracy.

    Supports both linear and RBF kernels. Iterates over combinations of C and gamma
    (if applicable), trains the model, and selects the best parameters based on validation accuracy.

    Parameters:
        X_train (ndarray): Training features
        y_train (ndarray): Training labels
        X_val (ndarray): Validation features
        y_val (ndarray): Validation labels
        C_values (list): List of C values to try
        gamma_values (list): List of gamma values (only for RBF)
        kernel_type (str): 'linear' or 'rbf'

    Returns:
        best_params (dict): Dictionary containing best C and (if applicable) gamma
        best_accuracy (float): Best validation accuracy achieved
    """
    best_params = None
    best_accuracy = 0

    if kernel_type == "linear":
        kernel = linear_kernel
        for C in tqdm(C_values, desc="Tuning C for linear kernel", ncols=80, position=0, leave=True):
            a, b = smo_algorithm(X_train, y_train, C, kernel=kernel)
            y_pred = predict(X_train, y_train, X_val, a, b, kernel)
            accuracy = accuracy_score(y_val, y_pred)
            if accuracy > best_accuracy: # Chose the best accuracy
                best_accuracy = accuracy
                best_params = {'C': C}
    elif kernel_type == "rbf" and gamma_values is not None: # Check each "C" with each gamma
        for C in tqdm(C_values, desc="Tuning C for RBF kernel", ncols=80, position=0, leave=True):
            for gamma in tqdm(gamma_values, desc=f"Tuning gamma for C={C}", ncols=80, position=1, leave=False):
                kernel = lambda x1, x2: rbf_kernel(x1, x2, gamma)
                a, b = smo_algorithm(X_train, y_train, C, kernel=kernel)
                y_pred = predict(X_train, y_train, X_val, a, b, kernel)
                accuracy = accuracy_score(y_val, y_pred)
                if accuracy > best_accuracy: # Chose the best accuracy
                    best_accuracy = accuracy
                    best_params = {'C': C, 'gamma': gamma}
    else:
        raise ValueError("Invalid kernel type or missing gamma values for RBF")

    return best_params, best_accuracy

# Function to train and tune models for each class
def train_and_tune_models(X, y):
    """
    Trains one-vs-rest binary SVM classifiers for each class using SMO.

    For each class in the dataset:
    - Relabels the dataset as binary (+1 vs -1)
    - Splits data into train, validation, and test
    - Tunes hyperparameters (C for linear, C and gamma for RBF)
    - Trains final models with best parameters and stores them

    Parameters:
        X (ndarray): Input features
        y (ndarray): Class labels

    Returns:
        binary_models_linear (list): List of trained models using linear kernel
        binary_models_rbf (list): List of trained models using RBF kernel
        X_train (ndarray): Last used training features
        y_train (ndarray): Last used training labels
        X_test (ndarray): Last used test features
        y_test (ndarray): Last used test labels
    """
    unique_classes = np.unique(y)
    binary_models_linear = []
    binary_models_rbf = []

    for class_label in tqdm(unique_classes, desc="\nTraining models for each class", ncols=80, position=0, leave=True):
        print(f"\nTraining model for class {class_label} vs others:")

        # Relabel the dataset: current class as +1, others as -1
        y_binary = np.where(y == class_label, 1, -1)

        # Split the dataset into train, validation, and test sets (60%, 20%, 20%)
        X_train, X_temp, y_train, y_temp = train_test_split(X, y_binary, test_size=0.4, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        # Tune the hyperparameters C for each kernel and gamma for RBF kernel
        C_values = [0.1, 1, 10]
        gamma_values = [0.01, 0.1, 1]  # Only used for RBF kernel

        # Tuning for linear kernel
        best_params_linear, best_accuracy_linear = tune_parameters(X_train, y_train, X_val, y_val, C_values, kernel_type="linear")

        # Tuning for RBF kernel
        best_params_rbf, best_accuracy_rbf = tune_parameters(X_train, y_train, X_val, y_val, C_values, gamma_values=gamma_values, kernel_type="rbf")

        # Print results of the two kernels
        print(f"\n--Best params for linear kernel for class {class_label}: {best_params_linear} (Validation Accuracy: {best_accuracy_linear})")
        print(f"--Best params for RBF kernel for class {class_label}: {best_params_rbf} (Validation Accuracy: {best_accuracy_rbf})\n")

        # Train the model with the best params for linear kernel
        kernel_linear = linear_kernel
        a_linear, b_linear = smo_algorithm(X_train, y_train, best_params_linear['C'], kernel=kernel_linear)

        # Store the model parameters for linear kernel
        binary_models_linear.append((a_linear, b_linear, best_params_linear['C']))

        # Train the model with the best params for RBF kernel
        kernel_rbf = lambda x1, x2: rbf_kernel(x1, x2, best_params_rbf['gamma'])
        a_rbf, b_rbf = smo_algorithm(X_train, y_train, best_params_rbf['C'], kernel=kernel_rbf)

        # Store the model parameters for RBF kernel
        binary_models_rbf.append((a_rbf, b_rbf, best_params_rbf['C'], best_params_rbf['gamma']))

    return binary_models_linear, binary_models_rbf, X_train, y_train, X_test, y_test

In [ ]:
# Load the Iris dataset
data = load_iris()
X, y = data.data, data.target

# Training phase
binary_models_linear, binary_models_rbf, X_train, y_train, X_test, y_test = train_and_tune_models(X, y)

In [7]:
# Visualization function
def visualize_models(binary_models_linear, binary_models_rbf, X, y):
    """
    Evaluates and prints performance of trained SVM models on the test set.

    For each class:
    - Relabels data for binary classification
    - Applies both linear and RBF models to the test set
    - Computes and displays confusion matrix, accuracy, and sensitivity

    Parameters:
        binary_models_linear (list): Trained models with linear kernel
        binary_models_rbf (list): Trained models with RBF kernel
        X (ndarray): Original features
        y (ndarray): Original class labels
    """
    unique_classes = np.unique(y)
    class_names = {0: "Setosa", 1: "Versicolor", 2: "Virginica"}  # Replace with actual class names if available

    for idx, class_label in enumerate(unique_classes):
        class_name = class_names.get(class_label, f"Class {class_label}")
        print(f"Results for {class_name} (Class {class_label}):")

        # Relabel the dataset: current class as +1, others as -1
        y_binary = np.where(y == class_label, 1, -1)

        # Split the dataset into train, validation, and test sets
        X_train, X_temp, y_train, y_temp = train_test_split(X, y_binary, test_size=0.4, random_state=42)
        X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

        # Evaluate the model for linear kernel
        a_linear, b_linear, C_linear = binary_models_linear[idx]
        y_test_pred_linear = predict(X_train, y_train, X_test, a_linear, b_linear, linear_kernel)
        conf_matrix_linear = confusion_matrix(y_test, y_test_pred_linear)
        accuracy_linear = accuracy_score(y_test, y_test_pred_linear)

        # Calculate TP, TN, FP, FN and sensitivity for linear kernel
        TP_linear = conf_matrix_linear[1, 1]
        TN_linear = conf_matrix_linear[0, 0]
        FP_linear = conf_matrix_linear[0, 1]
        FN_linear = conf_matrix_linear[1, 0]
        sensitivity_linear = TP_linear / (TP_linear + FN_linear) if (TP_linear + FN_linear) > 0 else 0

        print("Linear Kernel:")
        print("Confusion Matrix:")
        print(conf_matrix_linear)
        print(f"Test Accuracy: {accuracy_linear * 100:.2f}%")
        print("Table of Confusion:")
        print(f"TP={TP_linear}, TN={TN_linear}, FP={FP_linear}, FN={FN_linear}")
        print(f"Sensitivity: {sensitivity_linear * 100:.2f}%\n")

        # Evaluate the model for RBF kernel
        a_rbf, b_rbf, C_rbf, gamma_rbf = binary_models_rbf[idx]
        kernel_rbf = lambda x1, x2: rbf_kernel(x1, x2, gamma_rbf)
        y_test_pred_rbf = predict(X_train, y_train, X_test, a_rbf, b_rbf, kernel_rbf)
        conf_matrix_rbf = confusion_matrix(y_test, y_test_pred_rbf)
        accuracy_rbf = accuracy_score(y_test, y_test_pred_rbf)

        # Calculate TP, TN, FP, FN and sensitivity for RBF kernel
        TP_rbf = conf_matrix_rbf[1, 1]
        TN_rbf = conf_matrix_rbf[0, 0]
        FP_rbf = conf_matrix_rbf[0, 1]
        FN_rbf = conf_matrix_rbf[1, 0]
        sensitivity_rbf = TP_rbf / (TP_rbf + FN_rbf) if (TP_rbf + FN_rbf) > 0 else 0

        print("RBF Kernel:")
        print("Confusion Matrix:")
        print(conf_matrix_rbf)
        print(f"Test Accuracy: {accuracy_rbf * 100:.2f}%")
        print("Table of Confusion:")
        print(f"TP={TP_rbf}, TN={TN_rbf}, FP={FP_rbf}, FN={FN_rbf}")
        print(f"Sensitivity: {sensitivity_rbf * 100:.2f}%\n")

        print("-------------------")

# Visualization phase
visualize_models(binary_models_linear, binary_models_rbf, X, y)

Results for Setosa (Class 0):
Linear Kernel:
Confusion Matrix:
[[19  0]
 [ 0 11]]
Test Accuracy: 100.00%
Table of Confusion:
TP=11, TN=19, FP=0, FN=0
Sensitivity: 100.00%

RBF Kernel:
Confusion Matrix:
[[19  0]
 [ 0 11]]
Test Accuracy: 100.00%
Table of Confusion:
TP=11, TN=19, FP=0, FN=0
Sensitivity: 100.00%

-------------------
Results for Versicolor (Class 1):
Linear Kernel:
Confusion Matrix:
[[11  6]
 [ 0 13]]
Test Accuracy: 80.00%
Table of Confusion:
TP=13, TN=11, FP=6, FN=0
Sensitivity: 100.00%

RBF Kernel:
Confusion Matrix:
[[17  0]
 [ 1 12]]
Test Accuracy: 96.67%
Table of Confusion:
TP=12, TN=17, FP=0, FN=1
Sensitivity: 92.31%

-------------------
Results for Virginica (Class 2):
Linear Kernel:
Confusion Matrix:
[[24  0]
 [ 1  5]]
Test Accuracy: 96.67%
Table of Confusion:
TP=5, TN=24, FP=0, FN=1
Sensitivity: 83.33%

RBF Kernel:
Confusion Matrix:
[[24  0]
 [ 0  6]]
Test Accuracy: 100.00%
Table of Confusion:
TP=6, TN=24, FP=0, FN=0
Sensitivity: 100.00%

-------------------


# Summary

In this final section, we summarize and compare the classification results of the SVM models trained with linear and RBF kernels on the Iris dataset.

### Key Observations:

- **Setosa (Class 0):**  
  Both linear and RBF kernels achieved perfect classification, with 100% accuracy and sensitivity.

- **Versicolor (Class 1):**  
  - The **linear kernel** performed well with 80% accuracy and 100% sensitivity, but produced several false positives.  
  - The **RBF kernel** significantly improved performance, reaching 96.67% accuracy and 92.31% sensitivity.

- **Virginica (Class 2):**  
  - The **linear kernel** achieved 96.67% accuracy and 83.33% sensitivity.  
  - The **RBF kernel** again reached perfect performance: 100% accuracy and sensitivity.

### Conclusion:

The RBF kernel consistently outperformed or matched the linear kernel across all classes, particularly for more complex boundaries (e.g., Class 1 and 2).  
This demonstrates the power of the RBF kernel in handling non-linearly separable data in multi-class classification tasks.

